In [320]:
import numpy as np
import pandas as pd
import sklearn
import joblib
import sklearn.datasets as skd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

In [321]:
## Age ,
## Temperature ,
## loss of appetite

## sore throat,
## headache
## sweating

## Target  ====> Covid Disease.

In [322]:
rows = 25000

In [323]:
# Age.
age = np.random.randint(low = 10 , high = 90, size = rows )
age .shape

(25000,)

In [324]:
# temperature

temperature = []
for i in  range( rows ):
    val = np.random.randint( low = 97 , high = 105 )
    val2 = np.random.random()
    temperature.append( val  + val2 )

temperature = np.round( temperature , decimals = 5)

In [325]:
temperature.shape, np.unique( temperature).shape

((25000,), (24623,))

In [326]:
# loss of appetite
fatigue =  np.random.choice( a = ["Yes", "No"], size = rows )
fatigue.shape

(25000,)

In [327]:
## sore_throat

sore_throat =  np.random.choice( a = ["Yes", "No"], size = rows )
sore_throat.shape

(25000,)

In [328]:
## headache
headache = np.random.choice( a = ["Mild", "Severe"], size = rows )

print( headache.shape)

## cough
cough =  np.random.choice( a = ["Dry", "others"], size = rows )
print( cough.shape )

(25000,)
(25000,)


In [329]:
# gen_target = np.random.choice( a = [1, 2, 3, 4], size = 10000 )

gen_target =  np.random.choice( a = ["Covid", "Fever", "Migrane", 'Cholera'  ], size = rows )

In [473]:
## yellow eyes

eyes_color = np.array( (  [ 'Normal' ]* 20000  ) + [ 'yellow'] * 5000 )
print(eyes_color.shape)
np.random.shuffle(eyes_color )

(25000,)


In [474]:
hld_dat = {'Age' : age, 'Temperature' : temperature , 'Fatigue' : fatigue , 'Sore_Throat' :sore_throat,  'Eye Color' : eyes_color,
           'Headache': headache, 'Cough' : cough, 'Disease_type': gen_target  }

In [475]:
dat = pd.DataFrame( hld_dat )
dat

,Age,Temperature,Fatigue,Sore_Throat,Eye Color,Headache,Cough,Disease_type
0,82,102.16531,No,Yes,Normal,Severe,Dry,Cholera
1,43,103.92429,Yes,Yes,Normal,Severe,Dry,Fever
2,18,97.01768,No,Yes,Normal,Severe,Dry,Migrane
3,57,98.92275,No,No,Normal,Severe,Dry,Cholera
4,25,97.69337,Yes,No,Normal,Mild,Dry,Migrane
...,...,...,...,...,...,...,...,...
24995,83,102.67860,Yes,Yes,Normal,Severe,others,Cholera
24996,88,98.30138,Yes,Yes,Normal,Mild,Dry,Fever
24997,55,104.86730,No,Yes,Normal,Severe,Dry,Migrane
24998,21,101.81196,No,No,yellow,Mild,others,Fever


In [476]:
X = dat[ dat.columns[ : -1]]
X.head()

,Age,Temperature,Fatigue,Sore_Throat,Eye Color,Headache,Cough
0,82,102.16531,No,Yes,Normal,Severe,Dry
1,43,103.92429,Yes,Yes,Normal,Severe,Dry
2,18,97.01768,No,Yes,Normal,Severe,Dry
3,57,98.92275,No,No,Normal,Severe,Dry
4,25,97.69337,Yes,No,Normal,Mild,Dry


In [477]:
y = dat['Disease_type']
y.head()

0    Cholera
1      Fever
2    Migrane
3    Cholera
4    Migrane
Name: Disease_type, dtype: object

In [478]:
X_train, X_test, y_train, y_test = train_test_split( X, y , stratify = y ,  test_size = 0.2,  random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000, 7), (5000, 7), (20000,), (5000,))

In [479]:
num_var = dat.columns[ : 2].tolist()
cat_var = dat.columns[2: -1]
num_var, cat_var

(['Age', 'Temperature'],
 Index(['Fatigue', 'Sore_Throat', 'Eye Color', 'Headache', 'Cough'], dtype='object'))

In [480]:

numeric_features = num_var
numeric_transformer = Pipeline(steps=[ ('scaler', StandardScaler()  )  ]  )

categorical_features = cat_var
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer,  [ 0, 1] ),  # numeric_features
        ('cat', categorical_transformer, [2, 3, 4, 5, 6] ) ] )   #categorical_features

In [481]:
clf = Pipeline(steps=[('preprocessor', preprocessor), ('rf_clf', RandomForestClassifier(n_estimators = 1, max_depth = 10 ))])  

In [482]:

clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1]),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [2, 3, 4, 5, 6])])),
                ('rf_clf',
                 RandomForestClassifier(max_depth=10, n_estimators=1))])

In [483]:
y_train_pred = clf.predict( X_train  )
y_train_pred

array(['Cholera', 'Migrane', 'Covid', ..., 'Cholera', 'Migrane',
       'Migrane'], dtype=object)

In [484]:
accuracy_score( y_train, y_train_pred )

0.3291

In [485]:
y_test_pred = clf.predict( X_test )
y_test_pred

array(['Migrane', 'Covid', 'Cholera', ..., 'Cholera', 'Cholera', 'Covid'],
      dtype=object)

In [486]:
accuracy_score( y_test, y_test_pred )

0.2496

In [487]:
joblib.dump( clf, 'model.pkl' )

['model.pkl']

In [488]:
clf2 = joblib.load( 'model' )

In [489]:
print( X_test.iloc[0] )

Age                 42
Temperature    101.513
Fatigue            Yes
Sore_Throat         No
Eye Color       Normal
Headache          Mild
Cough              Dry
Name: 20876, dtype: object


In [490]:
q = np.array( [ 42, 101.513, 'Yes', 'No', 'Normal' , 'Mild', 'Dry' ] , dtype = 'object')
q = np.reshape(q, (-1, len(q)))

In [491]:
len(q)

1

In [492]:
############################################## TESTING CODE #####################################################
#  pd.DataFrame(q, columns = dat.columns[:-1]) 
# clf2.predict( X_test.iloc[:1] )

# clf2.predict(  pd.DataFrame(q, columns = dat.columns[:-1])   )

############################################## TESTING CODE #####################################################

In [493]:
clf2.predict( q  )

array(['Covid'], dtype=object)

In [494]:
clf.predict_proba( q )

array([[0.19339623, 0.25707547, 0.25235849, 0.29716981]])